In [1]:
using RDatasets, Test, GLM, LinearAlgebra, GeneralizedCopula
using LinearAlgebra: BlasReal, copytri!

# Dataframe with columns: Batch (Categorical), Yield (Int32)
dyestuff = dataset("lme4", "Dyestuff")
groups = unique(dyestuff[!, :Batch])
n, p, m = length(groups), 1, 1
d = Normal()
D = typeof(d)
gcs = Vector{GaussianCopulaVCObs{Float64, D}}(undef, n)
for (i, grp) in enumerate(groups)
    gidx = dyestuff[!, :Batch] .== grp
    ni = count(gidx)
    y = Float64.(dyestuff[gidx, :Yield])
    X = ones(ni, 1)
    V = [ones(ni, ni)]
    gcs[i] = GaussianCopulaVCObs(y, X, V, d)
end
gcm = GaussianCopulaVCModel(gcs);

In [2]:
# initialize β and τ from least square solution
@info "Initial point:"
initialize_model!(gcm);
#gcm.β .= [1527.4999999999998]
@show gcm.β
# update σ2 and τ from β using the MM algorithm
fill!(gcm.Σ, 1)
# update_Σ!(gcm, 500, 1e-6, GurobiSolver(OutputFlag=0), true)
update_Σ!(gcm)
@show gcm.τ
@show gcm.Σ;

┌ Info: Initial point:
└ @ Main In[2]:2


gcm.β = [1527.4999999999998]
gcm.τ = [0.00033164503710934656]
gcm.Σ = [0.8636149140056555]


In [3]:
# gc = gcm.data[1]
@test copula_gradient(gcm) ≈ [0.06561148856048]
@test copula_hessian(gcm) ≈ [-0.01964435497982633]

@show loglikelihood!(gcm, true, true)
@test loglikelihood!(gcm, true, true) ≈ loglikelihood2!(gcm, true, true)

loglikelihood!(gcm, true, true) = -164.0008237940362


Test Passed

In [4]:
# fit model using NLP on profiled loglikelihood
@info "MLE:"
# @time GeneralizedCopula.fit!(gcm, IpoptSolver(print_level=5))
@time GeneralizedCopula.fit!(gcm, IpoptSolver(print_level = 5, derivative_test = "first-order"))

┌ Info: MLE:
└ @ Main In[4]:2



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Starting derivative checker for first derivatives.


No errors detected by derivative checker.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bound

GaussianCopulaVCModel{Float64,Normal{Float64}}(GaussianCopulaVCObs{Float64,Normal{Float64}}[GaussianCopulaVCObs{Float64,Normal{Float64}}([1545.0, 1440.0, 1440.0, 1520.0, 1580.0], [1.0; 1.0; … ; 1.0; 1.0], [[1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.0 … 1.0 1.0]], [-0.009332475479925616], [-1.0; -1.0; … ; -1.0; -1.0], [-1224.6763131898897], [8.444616957336811e-11], [65.10568377994359], [2.3824856577e-314], [0.05966153756787458, -1.894018152216939, -1.894018152216939, -0.40550029333327153, 0.7108881008294792], [5.0], [2.2747256954e-314 2.274716976e-314 … 2.2747256954e-314 5.0e-324], [2.5], [5.858419862007839], [-3.4229869593697955, -3.4229869593697955, -3.4229869593697955, -3.4229869593697955, -3.4229869593697955], [2.5063658596e-314], [1541.7935063882878, 1541.7935063882878, 1541.7935063882878, 1541.7935063882878, 1541.7935063882878], [1541.7935063882878, 1541.7935063882878, 1541.7935063882878, 1541.7935063882878, 1541.7935063882878], [1.0, 1.0, 1.0, 1.0, 1.0], 

In [5]:
@show gcm.β
@show gcm.τ
@show gcm.Σ
@test copula_loglikelihood(gcm)[1] ≈ -163.35545423301846
@show loglikelihood2!(gcm, true, true)
@test loglikelihood2!(gcm, true, false) ≈ -163.35545423301846
@show gcm.∇β
@show gcm.∇τ
@show gcm.∇Σ

gcm.β = [1541.7935063882878]
gcm.τ = [0.0003462008462836904]
gcm.Σ = [52109.25787010273]
loglikelihood2!(gcm, true, true) = -163.3554542330147
gcm.∇β = [0.2969054404846284]
gcm.∇τ = [1.1571137292776257e-7]
gcm.∇Σ = [0.0]


1-element Array{Float64,1}:
 0.0